In [ ]:
%pylab inline

from PyTools import err, binner, nbins
from sklearn.cluster import DBSCAN
from scipy.optimize import leastsq
from string import join
from scipy.ndimage.filters import gaussian_filter
import pres_style, os

In [ ]:
def put_FITS_names(table):
    descr = table.dtype.descr
    for i in xrange(3): descr.pop(0)
    descr = [("input_file", "|S25")] + descr

    new_table = zeros(table.shape[0], dtype=descr)
    new_table["input_file"] = ["spSpec-{0:05d}-{1:04d}-{2:03d}.fit".format(r["mjd"], r["plate"], r["fiberID"]) for r in table]
    for f in new_table.dtype.names[1:]:
        new_table[f] = table[f]
    
    return new_table

par_lbs = [r"$M_\star$", r"$\left<\log{t_\star}\right>_M$", r"$\left<\log{t_\star}\right>_L$",
           r"$\left<\log{Z_\star/Z\odot}\right>_M$", r"$\left<\log{Z_\star/Z\odot}\right>_L$", r"$A_V$"]

if "paper_style" in dir():
    mnorm = 1e9
    od = os.path.expandvars("$HOME/Documents/Papers/photofit_near_paper/")
    par_lbs = [r"$\log{M_\star/M\odot}$", r"$\left<\log{t_\star}\right>_M$", r"$\left<\log{t_\star}\right>_L$",
               r"$\left<\log{Z_\star/Z\odot}\right>_M$", r"$\left<\log{Z_\star/Z\odot}\right>_L$", r"$A_V$",
               r"$A_V$", r"$z$"]
    res_lbs = [r"$\Delta\log{M_\star}$", r"$\Delta\left<\log{t_\star}\right>_M$",
               r"$\Delta\left<\log{t_\star}\right>_L$", r"$\Delta\left<\log{Z_\star}\right>_M$",
               r"$\Delta\left<\log{Z_\star}\right>_L$", r"$\Delta A_V$"]
    hatching = "/"*15
    gray = "0.3"
    ssag_cl = "0.2"
    sdss_cl = "#F6C141"
    sdss_sg = [sdss_cl, "#E8601C"]
    jpas_cl = "#90C987"
    jpas_sg = [jpas_cl, "#4EB265"]
    spec_cl = "#7BAFDE"
    spec_sg = [spec_cl, "#1965B0"]
    map_col = cm.viridis
else:
    mnorm = 1e8
    od = ""
    par_lbs = ["Mass (log/M$\odot$)", "Age (log/yr)", "Age (log/yr)", "Metallicity (log/Z$\odot$)",
               "Metallicity (log/Z$\odot$)", "Dust Extinction (mag)", "Redshift"]

In [ ]:
#READ TABLES

catalog = genfromtxt("catalog.txt", dtype=None, names=True)
spec_ne = genfromtxt("spec_ne.txt", dtype=None, names=True)
jpas_ye = genfromtxt("jpas_ye.txt", dtype=None, names=True)
jpas_ne = genfromtxt("jpas_ne.txt", dtype=None, names=True)
jpas_me = genfromtxt("jpas_me.txt", dtype=None, names=True)
gal2005 = genfromtxt(os.path.expandvars("$HOME/Programs/gallazzi+05/gallazzi_stats.log"), dtype=None, names=True, usecols=(0,1,2,8,14,20))

In [ ]:
#CLEAN GALLAZZI TABLE

# remove null values
mask = where(gal2005["m_mode"] != -99.0)
gal2005 = gal2005[mask]

# convert log(Z) into log(Z/Zo)
gal2005["z_mode"] -= log10(0.02)

# homologate field names
gal2005 = put_FITS_names(gal2005)
gal2005.dtype.names = ("input_file", "M_mod", "log_t_L_mod", "log_Z_L_mod")

In [ ]:
# match gal2005 & catalog
match_g05_in_cat = array([n in catalog["input_file"] for n in gal2005["input_file"]], dtype=bool)
match_cat_in_g05 = array([n in gal2005["input_file"] for n in catalog["input_file"]], dtype=bool)

In [ ]:
# sort by input_file
g05_sort = argsort(gal2005["input_file"][match_g05_in_cat])
cat_sort = argsort(catalog["input_file"][match_cat_in_g05])

In [ ]:
print all(catalog["input_file"][match_cat_in_g05]==gal2005["input_file"][match_g05_in_cat])